<a href="https://colab.research.google.com/github/debu-sinha/building-ai-agents/blob/main/Product_Q_A_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install all dependency packages
#Remember to execute this before running any of the exercises

!pip install langchain
!pip install langchain-openai
!pip install langchain_community
!pip install langchain_chroma
!pip install langgraph

!pip install tenacity
!pip install pysqlite3-binary
!pip install pandas
!pip install pypdf
!pip install nbformat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Setup Models

In [2]:
from google.colab import userdata

from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

from langchain_openai import OpenAIEmbeddings, ChatOpenAI

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=userdata.get('oai'))
chat_model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=userdata.get("oai"))

In [4]:
import pandas as pd

product_pricing_df = pd.read_csv("./data/Laptop pricing.csv")
product_pricing_df.head()

,Name,Price,ShippingDays
0,AlphaBook Pro,1499,2
1,GammaAir X,1399,7
2,SpectraBook S,2499,7
3,OmegaPro G17,2199,14
4,NanoEdge Flex,1699,2


### Add Product Pricing function tool

In [5]:
@tool
def get_laptop_price(laptop_name: str) -> int:
  """
  This function return the price of a laptop given its name.
  It performs a substring match between input name and the name of the laptop.
  If a match is found it returns the price of the laptop.
  If no match is found it returns -1.
  """
  match_records_df = product_pricing_df[product_pricing_df["Name"].str.contains("^" + laptop_name, case=False)]
  if len(match_records_df) > 0:
    return match_records_df["Price"].iloc[0]
  else:
    return -1

In [22]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Add Product Features Retrieval Tool

In [14]:
from langchain_chroma import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

loader = PyPDFLoader("./data/Laptop product descriptions.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

prod_feature_store = Chroma.from_documents(splits, embedding_model)

get_product_features = create_retriever_tool(prod_feature_store.as_retriever(search_kwargs={"k": 2}),
                                             name="Get_Product_Features",
                                             description="""This store contains details about Laptops. It lists the available laptops and theor features
                                             including CPU, memory, storage, design and advantages
                                             """
                                             )

### Setup a Product QnA chatbot

In [15]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

tools = [get_product_features, get_laptop_price]

checkpointer = MemorySaver()

product_qna_agent = create_react_agent(model=chat_model, tools=tools, prompt=system_prompt, checkpointer=checkpointer)

### To maintain memory, each request should be in the context of a thread.
### Each user conversation will use a separate thread ID

In [17]:
import uuid

config = {"configurable": {"thread_id": uuid.uuid4()}}

inputs = {"messages": [("user", "Whate are the features of GammaAir?")]}

for stream in product_qna_agent.stream(inputs, config=config, stream_mode="values"):
  message = stream["messages"][-1]
  if isinstance(message, tuple):
    print(message)
  else:
    message.pretty_print()

================================ Human Message =================================

Whate are the features of GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_KWEtk7jf0u4xxEjZtLFDOMUw)
 Call ID: call_KWEtk7jf0u4xxEjZtLFDOMUw
  Args:
    query: GammaAir
================================= Tool Message =================================
Name: Get_Product_Features

Fictional Laptop Descriptions
AlphaBook Pro
The AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,
and a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of
power and portability.
GammaAir X
GammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB
NVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a
portable design.
SpectraBook S

Fictional Laptop Descriptions
AlphaBook Pro
The AlphaBook Pro is a sleek ult

### Execute the Product QnA Chatbot

In [19]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_qna_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : That's great! What specifications or features are you looking for in a laptop?
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here are some available laptop names:

1. **AlphaBook Pro** - A sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM, and a fast 1TB SSD.
   
2. **GammaAir X** - Combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB NVMe SSD.

3. **SpectraBook S** - [Details about this model are not provided, but it is available.]

If you would like more information on any specific laptop or need help deciding, just let me know!
----------------------------------------
USER : Tell me about the features of  SpectraBook
AGENT : The **SpectraBook S** is designed for power users and offers the following features:

- **Proc